In [2]:
from ultralytics import YOLO
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# Path to dataset in Google Drive
DATA_ROOT = Path(r"rumex_dataset")

train_images = DATA_ROOT / "train/images"
valid_images = DATA_ROOT / "valid/images"
test_images  = DATA_ROOT / "test/images"

train_labels = DATA_ROOT / "train/labels"
valid_labels = DATA_ROOT / "valid/labels"
test_labels  = DATA_ROOT / "test/labels"

print("DATA_ROOT:", DATA_ROOT)
print("Train images exist:", train_images.exists())
print("Valid images exist:", valid_images.exists())
print("Test images exist:",  test_images.exists())
print("Train labels exist:", train_labels.exists())
print("Valid labels exist:", valid_labels.exists())
print("Test labels exist:",  test_labels.exists())

DATA_ROOT: rumex_dataset
Train images exist: True
Valid images exist: True
Test images exist: True
Train labels exist: True
Valid labels exist: True
Test labels exist: True


## Train YOLOv11s

In [4]:
MODEL_NAME = "yolo11s.pt"
model = YOLO(MODEL_NAME)
DATA_YAML_PATH = DATA_ROOT / "data.yaml"

results = model.train(
    data=str(DATA_YAML_PATH),
    lr0=0.02,
    epochs=120,
    imgsz=640,
    batch=8,
    #patience=10,
)

results

New https://pypi.org/project/ultralytics/8.3.232 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.229  Python-3.13.0 torch-2.9.1+cu130 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=rumex_dataset\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=120, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.02, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, ops

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001BC134DFF50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

## Plot progress over epochs

In [22]:
"""
Plots mAP score improvement over each epoch
"""

iterationRun = r"runs/detect/train/results.csv"

epochs = []
mAP = []
with open(iterationRun, "r", encoding="utf-8") as infile:
    infile.readline()
    for lines in infile:
        splitLines = lines.split(",")
        epochs.append(int(splitLines[0]))
        mAP.append(float(splitLines[8]))

plt.plot(epochs, mAP)
plt.xlabel("Epochs")
plt.ylabel("mAP50-95")
plt.yticks(np.arange(0, 1, 0.05))
plt.show()


<Figure size 640x480 with 1 Axes>

## Evaluate on Test Set

In [ ]:
model = YOLO(r"runs\detect\Old-dataset\epoch500\weights\best.pt")
metrics_valid = model.val(data=str(DATA_YAML_PATH), split="test", imgsz=640)
metrics_valid

Ultralytics 8.3.229  Python-3.13.0 torch-2.9.1+cu130 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
YOLO11s summary (fused): 100 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 3098.0197.7 MB/s, size: 3840.6 KB)
val: Scanning C:\Users\Ludvi\Documents\INF250_NMBU\Assignment-3\rumex_dataset\test\labels.cache... 33 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 33/33 101.9Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 1.5s/it 4.5s3.3s0s
                   all         33       1835      0.706      0.689      0.727      0.331
Speed: 3.4ms preprocess, 34.2ms inference, 0.0ms loss, 5.6ms postprocess per image
Results saved to C:\Users\Ludvi\Documents\INF250_NMBU\Assignment-3\runs\detect\val2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001BC56F4E270>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

## Export predictions_test.csv

In [ ]:
import csv

csv_path = DATA_ROOT / "predictions_test.csv"
rows = []

results_pred = model.predict(
    source=str(test_images),
    imgsz=640,
    conf=0.25,
    save=False,
    verbose=False
)

for r in results_pred:
    img_path = Path(r.path)
    h, w = r.orig_shape

    if r.boxes is None:
        continue

    for b in r.boxes:
        cls_id = int(b.cls.item()) if b.cls is not None else 0
        conf = float(b.conf.item()) if b.conf is not None else 0.0
        x1, y1, x2, y2 = b.xyxy[0].tolist()

        bw = (x2 - x1) / w
        bh = (y2 - y1) / h
        xc = (x1 + x2) / 2 / w
        yc = (y1 + y2) / 2 / h

        rows.append([
            img_path.name,
            CLASS_NAMES[cls_id],
            f"{conf:.4f}",
            f"{xc:.6f}",
            f"{yc:.6f}",
            f"{bw:.6f}",
            f"{bh:.6f}"
        ])

with open(csv_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["image", "class", "conf", "x_center", "y_center", "width", "height"])
    writer.writerows(rows)

print("Wrote:", csv_path)
print("Number of predicted boxes:", len(rows))